In [1]:
# Script to clean uspatentcitation.tsv

# Feb 7th, 2020
# I am adding a cut for citation date
# it cannot be older than 07-31-1790 (the first patent ever issued)

# Jan 20th, 2020
# There are citation_ids larger than 7 characters and smaller than 4
# Larger are usually applications, smallers tend to be errors
# I am keeping than so calculations on forward citations are accurate
# When matching by citation_id, it must be previously filtered

# as of Jan 9th, 2020, there are entries to be evaluated
# for now, error_bad_lines=False skips those entries


In [2]:
import sys
sys.path.append('/home/rkogeyam/scripts/')
from sampler import sampler

import pandas as pd
import numpy as np
import re
import datetime

In [3]:
# uspatentcitation.tsv
# uuid:         unique id
# patent_id:    patent number
# citation_id:  identifying number of patent to which select patent cites
# date:         date select patent (patent_id) cites patent (citation_id)
# name:         name of cited record
# kind:         WIPO document kind codes 
#               (http://www.uspto.gov/learning-and-resources/support-centers/electronic-business-center/kind-codes-included-uspto-patent)	2002 and After
# country:      country cited patent was granted (always US)
# category:     who cited the patent (examiner, applicant, other etc) - 2002 and After
# sequence:     order in which this reference is cited by select patent	all


In [4]:
src= r'/home/rkogeyam/PATENT_CITATION/data/uspatentcitation.tsv'
dst= '/home/rkogeyam/PATENT_CITATION/data/cleanuspatentcitation.csv'

In [5]:
df=pd.read_csv(src, sep='\t', error_bad_lines=False, quoting = csv.QUOTE_NONNUMERIC)

In [6]:
df=df.astype(str)

In [7]:
df.dtypes

uuid           object
patent_id      object
citation_id    object
date           object
name           object
kind           object
country        object
category       object
sequence       object
dtype: object

In [8]:
%%time
#stripping trailing white spaces - no changes
df['patent_id'] = df['patent_id'].str.strip()
df['citation_id'] = df['citation_id'].str.strip()

CPU times: user 1min 22s, sys: 18 ms, total: 1min 22s
Wall time: 1min 22s


In [9]:
%%time
# Keep this for reference!
# As of Dec 31st, 2019, I compared the clean to the raw version of citation and patent ids

# stripping non-desired characters but keeping the originals for later check - only three changes in citation_id

df['citation_id_raw']=df['citation_id']
df['patent_id_raw']=df['patent_id']

CPU times: user 1.87 s, sys: 6 ms, total: 1.88 s
Wall time: 1.87 s


In [10]:
%%time
cleaning_patent=lambda x:re.sub('([^a-zA-Z0-9]+)', "", x)
df['citation_id']=df['citation_id'].apply(cleaning_patent)

# # #this is taking a lot of time, evaluate alternatives

CPU times: user 4min 37s, sys: 38 ms, total: 4min 37s
Wall time: 4min 37s


In [11]:
%%time
df['patent_id']=df['patent_id'].apply(cleaning_patent)


CPU times: user 4min 37s, sys: 31 ms, total: 4min 37s
Wall time: 4min 37s


In [12]:
df.date.replace({'-00':'-01'}, regex=True, inplace=True)

In [13]:
%%time
df['date']=pd.to_datetime(df['date'], format="%Y-%m-%d", errors='coerce') 
# first_patent = datetime.date(1790, 7, 31)

CPU times: user 39.7 s, sys: 1.44 s, total: 41.1 s
Wall time: 41 s


In [14]:
first_patent = pd.to_datetime('1790-06-30', format="%Y-%m-%d") #I tweaked slightly because there is a citation to the patent n1 that seems correct

In [15]:
#as for Aug 11, 2020, two problems arose:
#- some name fields present EOF characters
#- some lines are missing their linebreaks
# this is the correction of these issues
clean_doublespace=lambda x:re.sub('\s\s', '\s', x)
df['name']=df['name'].apply(clean_doublespace)


In [16]:
#citation with dates previous to the inauguration of USPTO office
df[df['date']<first_patent][['patent_id','citation_id','date']]

,patent_id,citation_id,date
2097572,4865278,4161717,1779-07-01
7840155,5263595,4629077,1686-12-01
8713651,7224106,3832580,1774-08-01
9384584,5341713,3422714,1696-01-01
11408356,5159488,4725898,1688-02-01
13038551,6878571,3746934,1773-07-01
13419413,4976377,3855458,1774-12-01
16205263,4586322,4476632,1698-10-01
22361208,D297377,3513500,1790-05-01
22669196,5819410,5174519,1754-05-01


In [17]:
len(df['date']) - df['date'].count()

1772

In [18]:
%%time
#patents with wrong dates should not be dropped
#the best way is to search for the correct date
#i can do in a separated script
#this would improve the results - other citation dates can be bugged

#for now, because there are just a few mistakes, i just exclude the date
#excluding dates from patents with wrong dates
df['date']=df['date'].apply(lambda x: np.nan if x < first_patent else x)

CPU times: user 10min 11s, sys: 7.45 s, total: 10min 19s
Wall time: 10min 18s


In [19]:
len(df['date']) - df['date'].count()

1805

In [20]:
# print(df[df['patent_id']!=df['patent_id_raw']][['patent_id','patent_id_raw']])

In [21]:
# print(df[df['citation_id']!=df['citation_id_raw']][['citation_id', 'citation_id_raw']])

In [22]:
df[df['citation_id'].apply(lambda x: len(x)>7)]

,uuid,patent_id,citation_id,date,name,kind,country,category,sequence,citation_id_raw,patent_id_raw
729,000b5qt0y57vvnmr91ip5wr60,6873539,20030146488,2003-08-01,Nagano et al.,A1,US,cited by other,116,20030146488,6873539
1103,000gp68o7om42w13fl59sn9l1,10503926,10019339,2018-07-01,Von Hanxleden et al.,B2,US,cited by applicant,310,10019339,10503926
1372,000l397rmcu1p9fxvegt75gd8,6834474,20020090489,2002-07-01,Dobreski et al.,A1,US,cited by other,192,2002/0090489,6834474
1786,000qx88kavha2g9vje18ys9em,10568624,10376263,2019-08-01,Morgan et al.,B2,US,cited by applicant,4720,10376263,10568624
1929,000tgpcixodfh8fiv1ov4aiw7,6856112,20030160510,2003-08-01,Mizutani et al.,A1,US,cited by examiner,3,20030160510,6856112
2900,0018fat56bn962jnsrm2ivw7q,10314582,10028761,2018-07-01,Leimbach et al.,B2,US,cited by applicant,1613,10028761,10314582
3301,001eouu9s7r51sgah57ozbhn9,10482251,10185827,2019-01-01,Hamlin,B1,US,cited by examiner,5,10185827,10482251
3921,001oar7hv0dk87fsvk25symwb,10561422,10245058,2019-04-01,Omori et al.,B2,US,cited by applicant,4607,10245058,10561422
4806,0021zkyqbzv8dp89k0e9i5pvn,D474084,20010013500,2001-08-01,Gilley et al.,A1,US,cited by examiner,26,2001/0013500,D474084
4853,0022qzj7k5abu14rmzg71nz3r,6860140,20020109577,2002-08-01,Loose et al.,A1,US,cited by other,57,20020109577,6860140


In [23]:
df[df['citation_id'].apply(lambda x: len(x)<4)]

,uuid,patent_id,citation_id,date,name,kind,country,category,sequence,citation_id_raw,patent_id_raw
118900,01exzafox6dkx9x816ch4ohto,7459043,H72,1986-06-01,Wise et al.,H,US,cited by other,12,H72,7459043
328944,03wkd98jd0lodjm1r5kvkev2t,6402703,H93,1986-07-01,Matta et al.,H,US,cited by other,78,H93,6402703
964087,0bgpq77n77f6i6u0om5hdcy1t,6687642,H81,1986-07-01,Szabo et al.,H,US,cited by examiner,0,H81,6687642
974595,0bl6ucugzjdhtn6my7kohqhfq,10407790,H36,1986-03-01,Smith,H,US,cited by examiner,0,H36,10407790
1140247,0dkamzf60pcpz0bhks51azg85,4248123,84,NaT,nan,nan,US,nan,0,84,4248123
1503220,0hvm59r3v9ju20cu8ix2dqig2,9784096,H55,1986-05-01,Ramsey,H,US,cited by examiner,0,H55,9784096
1696649,0k6i32yssjvp5pskoa9ouc2u4,7951236,1,1836-07-01,Ruggles,A,US,cited by other,0,1,7951236
1702722,0k93nnz5wfwokh3rsqnyllce2,4696154,985,1838-10-01,Grieb,nan,US,nan,0,985,4696154
1709226,0kbu63dmdxy6aja48p4u7qw90,6082307,997,1838-11-01,Sanford,nan,US,nan,0,997,6082307
1781031,0l6n5jr3ssncd7b48a0l6qhu6,10342646,H83,1986-07-01,Poletto et al.,H,US,cited by applicant,9,H83,10342646


In [24]:
df.citation_id.str.len().value_counts()

7     107851612
6        767885
11       122970
8        109091
5         51054
4          7962
9          2440
3           646
2            67
1            42
12           40
10           25
15            1
13            1
Name: citation_id, dtype: int64

In [25]:
df.to_csv(dst)